In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import sys
import time
import sklearn
import tensorflow as tf
from tensorflow import keras
import pandas as pd

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd,  sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.1.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.2.0
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2.post1
tensorflow 2.1.0
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state =7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state =11)

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(3870, 8) (3870,)
(11610, 8) (11610,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)


In [5]:
# RandomizedSearchCV
# 1. transform to sklearn model
# 2. definite parameter collections
# 3. search parameters
def build_model(hidden_layers = 1,
                layer_size = 30,
                learning_rate = 3e-3):

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(layer_size, activation = 'relu', input_shape = x_train.shape[1:]))
    for _ in range(hidden_layers-1):
        model.add(keras.layers.Dense(layer_size, activation = 'relu'))
    out_put = model.add(keras.layers.Dense(1))

    optimizer = keras.optimizers.SGD(learning_rate)
    model.compile(loss = 'mse', optimizer = optimizer)
    return model

sklearn_model = keras.wrappers.scikit_learn.KerasRegressor(build_model)
callbacks = [keras.callbacks.EarlyStopping(patience = 5,
                                           min_delta = 1e-2)]

In [7]:
from scipy.stats import reciprocal
param_distribution = {
    'hidden_layers':[1, 2, 3, 4],
    'layer_size':np.arange(0, 100),
    'learning_rate':reciprocal(1e-4, 1e-2)
}

from sklearn.model_selection import RandomizedSearchCV
randomized_search_cv = RandomizedSearchCV(sklearn_model,
                                          param_distribution,
                                         )
randomized_search_cv.fit(x_train_scaled, y_train,
                         epochs = 100,
                         validation_data = [x_valid_scaled, y_valid],
                         callbacks = callbacks)

Train on 9288 samples, validate on 3870 samples
Epoch 1/100
9288/9288 [==============================] - 1s 89us/sample - loss: 1.4594 - val_loss: 0.7921
Epoch 2/100
9288/9288 [==============================] - 0s 41us/sample - loss: 0.7413 - val_loss: 0.6710
Epoch 3/100
9288/9288 [==============================] - 0s 51us/sample - loss: 0.6019 - val_loss: 0.6038
Epoch 4/100
9288/9288 [==============================] - 0s 42us/sample - loss: 0.5540 - val_loss: 0.5689
Epoch 5/100
9288/9288 [==============================] - 0s 43us/sample - loss: 0.5257 - val_loss: 0.5415
Epoch 6/100
9288/9288 [==============================] - 0s 41us/sample - loss: 0.5086 - val_loss: 0.5195
Epoch 7/100
9288/9288 [==============================] - 0s 50us/sample - loss: 0.4894 - val_loss: 0.5087
Epoch 8/100
9288/9288 [==============================] - 0s 46us/sample - loss: 0.4785 - val_loss: 0.4982
Epoch 9/100
9288/9288 [==============================] - 0s 47us/sample - loss: 0.4724 - val_loss: 0.486

9288/9288 [==============================] - 0s 40us/sample - loss: 0.4330 - val_loss: 0.4512
Epoch 16/100
9288/9288 [==============================] - 0s 37us/sample - loss: 0.4316 - val_loss: 0.4557
Epoch 17/100
9288/9288 [==============================] - 0s 39us/sample - loss: 0.4263 - val_loss: 0.4496
Epoch 18/100
9288/9288 [==============================] - 0s 39us/sample - loss: 0.4223 - val_loss: 0.4428
Epoch 19/100
9288/9288 [==============================] - 0s 40us/sample - loss: 0.4218 - val_loss: 0.4423
Epoch 20/100
9288/9288 [==============================] - 0s 39us/sample - loss: 0.4176 - val_loss: 0.4386
Epoch 21/100
9288/9288 [==============================] - 0s 38us/sample - loss: 0.4114 - val_loss: 0.4356
Epoch 22/100
9288/9288 [==============================] - 0s 36us/sample - loss: 0.4098 - val_loss: 0.4324
Epoch 23/100
9288/9288 [==============================] - 0s 38us/sample - loss: 0.4101 - val_loss: 0.4257
Epoch 24/100
9288/9288 [==========================

9288/9288 [==============================] - 0s 42us/sample - loss: 0.3966 - val_loss: 0.4175
Epoch 23/100
9288/9288 [==============================] - 0s 42us/sample - loss: 0.3960 - val_loss: 0.4156
Epoch 24/100
9288/9288 [==============================] - 0s 42us/sample - loss: 0.3930 - val_loss: 0.4158
Epoch 25/100
9288/9288 [==============================] - 0s 42us/sample - loss: 0.3898 - val_loss: 0.4121
Epoch 26/100
9288/9288 [==============================] - 0s 42us/sample - loss: 0.3892 - val_loss: 0.4174
Epoch 27/100
9288/9288 [==============================] - 0s 43us/sample - loss: 0.3875 - val_loss: 0.4077
Epoch 28/100
9288/9288 [==============================] - 0s 45us/sample - loss: 0.3886 - val_loss: 0.4080
Epoch 29/100
9288/9288 [==============================] - 0s 40us/sample - loss: 0.3906 - val_loss: 0.4058
Epoch 30/100
2322/2322 [==============================] - 0s 22us/sample - loss: 0.4119
Train on 9288 samples, validate on 3870 samples
Epoch 1/100
9288/9288

9288/9288 [==============================] - 0s 43us/sample - loss: 0.3595 - val_loss: 0.3805
Epoch 10/100
9288/9288 [==============================] - 0s 41us/sample - loss: 0.3532 - val_loss: 0.3716
Epoch 11/100
9288/9288 [==============================] - 0s 42us/sample - loss: 0.3471 - val_loss: 0.3709
Epoch 12/100
9288/9288 [==============================] - 0s 43us/sample - loss: 0.3426 - val_loss: 0.3621
Epoch 13/100
9288/9288 [==============================] - 0s 41us/sample - loss: 0.3388 - val_loss: 0.3608
Epoch 14/100
9288/9288 [==============================] - 0s 41us/sample - loss: 0.3353 - val_loss: 0.3551
Epoch 15/100
9288/9288 [==============================] - 0s 44us/sample - loss: 0.3314 - val_loss: 0.3820
Epoch 16/100
9288/9288 [==============================] - 0s 41us/sample - loss: 0.3294 - val_loss: 0.3542
Epoch 17/100
9288/9288 [==============================] - 0s 41us/sample - loss: 0.3231 - val_loss: 0.3458
Epoch 18/100
9288/9288 [==========================

9288/9288 [==============================] - 0s 39us/sample - loss: 0.4643 - val_loss: 0.4870
Epoch 34/100
9288/9288 [==============================] - 0s 40us/sample - loss: 0.4596 - val_loss: 0.4822
Epoch 35/100
9288/9288 [==============================] - 0s 40us/sample - loss: 0.4554 - val_loss: 0.4778
Epoch 36/100
9288/9288 [==============================] - 0s 41us/sample - loss: 0.4513 - val_loss: 0.4729
Epoch 37/100
9288/9288 [==============================] - 0s 43us/sample - loss: 0.4473 - val_loss: 0.4689
Epoch 38/100
9288/9288 [==============================] - 0s 42us/sample - loss: 0.4437 - val_loss: 0.4648
Epoch 39/100
9288/9288 [==============================] - 0s 40us/sample - loss: 0.4399 - val_loss: 0.4609
Epoch 40/100
9288/9288 [==============================] - 0s 38us/sample - loss: 0.4364 - val_loss: 0.4570
Epoch 41/100
9288/9288 [==============================] - 0s 40us/sample - loss: 0.4331 - val_loss: 0.4534
Epoch 42/100
9288/9288 [==========================

...

9288/9288 [==============================] - 0s 42us/sample - loss: 0.3968 - val_loss: 0.4052
Epoch 5/100
9288/9288 [==============================] - 0s 42us/sample - loss: 0.3806 - val_loss: 0.4290
Epoch 6/100
9288/9288 [==============================] - 0s 43us/sample - loss: 0.3718 - val_loss: 0.3778
Epoch 7/100
9288/9288 [==============================] - 0s 43us/sample - loss: 0.3624 - val_loss: 0.3916
Epoch 8/100
9288/9288 [==============================] - 0s 42us/sample - loss: 0.3554 - val_loss: 0.3809
Epoch 9/100
9288/9288 [==============================] - 0s 40us/sample - loss: 0.3497 - val_loss: 0.3667
Epoch 10/100
9288/9288 [==============================] - 0s 41us/sample - loss: 0.3443 - val_loss: 0.3691
Epoch 11/100
9288/9288 [==============================] - 0s 41us/sample - loss: 0.3379 - val_loss: 0.3527
Epoch 12/100
9288/9288 [==============================] - 0s 46us/sample - loss: 0.3349 - val_loss: 0.3558
Epoch 13/100
9288/9288 [==============================]

9288/9288 [==============================] - 0s 38us/sample - loss: 0.4738 - val_loss: 0.4915
Epoch 22/100
9288/9288 [==============================] - 0s 39us/sample - loss: 0.4708 - val_loss: 0.4908
Epoch 23/100
9288/9288 [==============================] - 0s 37us/sample - loss: 0.4681 - val_loss: 0.4860
Epoch 24/100
9288/9288 [==============================] - 0s 38us/sample - loss: 0.4656 - val_loss: 0.4814
Epoch 25/100
9288/9288 [==============================] - 0s 37us/sample - loss: 0.4630 - val_loss: 0.4790
Epoch 26/100
9288/9288 [==============================] - ETA: 0s - loss: 0.462 - 0s 38us/sample - loss: 0.4607 - val_loss: 0.4764
Epoch 27/100
9288/9288 [==============================] - 0s 37us/sample - loss: 0.4584 - val_loss: 0.4726
Epoch 28/100
9288/9288 [==============================] - 0s 38us/sample - loss: 0.4565 - val_loss: 0.4702
Epoch 29/100
9288/9288 [==============================] - 0s 36us/sample - loss: 0.4549 - val_loss: 0.4688
Epoch 30/100
9288/9288 [==

9288/9288 [==============================] - 0s 39us/sample - loss: 0.5673 - val_loss: 0.6159
Epoch 16/100
9288/9288 [==============================] - 0s 37us/sample - loss: 0.5573 - val_loss: 0.6036
Epoch 17/100
9288/9288 [==============================] - 0s 38us/sample - loss: 0.5482 - val_loss: 0.5936
Epoch 18/100
9288/9288 [==============================] - 0s 37us/sample - loss: 0.5396 - val_loss: 0.5842
Epoch 19/100
9288/9288 [==============================] - 0s 37us/sample - loss: 0.5315 - val_loss: 0.5756
Epoch 20/100
9288/9288 [==============================] - 0s 37us/sample - loss: 0.5244 - val_loss: 0.5669
Epoch 21/100
9288/9288 [==============================] - 0s 39us/sample - loss: 0.5175 - val_loss: 0.5591
Epoch 22/100
9288/9288 [==============================] - 0s 39us/sample - loss: 0.5111 - val_loss: 0.5525
Epoch 23/100
9288/9288 [==============================] - 0s 39us/sample - loss: 0.5052 - val_loss: 0.5455
Epoch 24/100
9288/9288 [==========================

9288/9288 [==============================] - 0s 51us/sample - loss: 0.7184 - val_loss: 0.7741
Epoch 6/100
9288/9288 [==============================] - 0s 49us/sample - loss: 0.6952 - val_loss: 0.7509
Epoch 7/100
9288/9288 [==============================] - 0s 50us/sample - loss: 0.6763 - val_loss: 0.7314
Epoch 8/100
9288/9288 [==============================] - 0s 48us/sample - loss: 0.6591 - val_loss: 0.7144
Epoch 9/100
9288/9288 [==============================] - 0s 53us/sample - loss: 0.6433 - val_loss: 0.6975
Epoch 10/100
9288/9288 [==============================] - 0s 49us/sample - loss: 0.6292 - val_loss: 0.6822
Epoch 11/100
9288/9288 [==============================] - 0s 49us/sample - loss: 0.6167 - val_loss: 0.6695
Epoch 12/100
9288/9288 [==============================] - 0s 49us/sample - loss: 0.6051 - val_loss: 0.6584
Epoch 13/100
9288/9288 [==============================] - 0s 50us/sample - loss: 0.5940 - val_loss: 0.6469
Epoch 14/100
9288/9288 [==============================

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fa7d05ef0d0>, as the constructor either does not set or modifies parameter layer_size

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize = (8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
    
plot_learning_curves(history)

In [ ]:
# f(x) = 1/(x*log(b/a)) a <= x <= b
from scipy.stats import reciprocal
reciprocal.rvs(1e-4, 1e-2, size = 10)